<a href="https://colab.research.google.com/github/kgautam2103/zoom_participant_list_public/blob/main/modular_code__zoom_participant_list_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import drive
drive.mount("mnt",force_remount=True)
%run '/content/mnt/MyDrive/Colab Notebooks/zoom_participant_list_utility_functions.ipynb'


Mounted at mnt


In [ ]:
import requests
import random
import json
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import gspread
import re
from pandas import ExcelWriter
from google.colab import files
from gspread_dataframe import set_with_dataframe
from pandas import json_normalize
from urllib.parse import urlencode
!pip install python-Levenshtein
!pip install fuzzywuzzy
from fuzzywuzzy import process


In [ ]:
report_date= input("enter the report date like 2021-07-11 : ")
move2side_input=input('move2side needed ? Enter true or false: ').lower()
#report_date = '2021-07-18'
apac_report_datetime = str(report_date)+'T02'
naol_report_datetime = str(report_date)+'T14'
meetingID = '9181716151'
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'
poll1_id= 'l1gX0McLRc6mxffk37nnIw'
poll2_id = 'mlE7n7f3TQ-qgXoF--MKPQ'


#to get result for the last held meeting
report_meetings = base_url+'report/meetings/'
#get_meeting_url = report_meetings+meetingID

get_meeting_url = past_meetings+meetingID+'/instances'
get_user_url = base_url+'users/'
apac_uid=''
naol_uid=''
page_size ='300'
apac_participants_list = []
naol_participants_list = []
apac_next_page_token = ''
naol_next_page_token = ''


current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_xlsx = current_ts+'_participants_list_'+report_date+".xlsx"

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WhACIU6F7i9VwDxXoqzx6GEwutQI5HpIYl4v6H49-7Q/edit#gid=0')

input_sheet = wb.worksheet('sahajyogis')
final_combined_sheet = wb.worksheet('combined_sheet')
rolling_3week_sheet = wb.worksheet('Rolling3Weekend')
weekend_archive_sheet = wb.worksheet('WeekendArchive')
Sideroom_sheet = wb.worksheet('regular_sideroom')
move2side_sheet = wb.worksheet('move2side')
updated_sideroom_sheet = wb.worksheet('updated_sideroom')
month_3_archive_frequency_sheet = wb.worksheet('3month_archive_frequency')

inputdata_df = read_worksheet(input_sheet,['id','name','email'])
rolling_3week_df = read_worksheet(rolling_3week_sheet,['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
weekend_archive_df = read_worksheet(weekend_archive_sheet,['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
sideroom_data_df = read_worksheet(Sideroom_sheet,['name'])


report_datetime = datetime.strptime(report_date, '%Y-%m-%d')
report_datetime_3week_before = report_datetime-timedelta(days=15)
report_datetime_3month_before = report_datetime-timedelta(days=90)

ref_id_list = list(filter(None, inputdata_df['id'].tolist()))
ref_name_list = list(filter(None, inputdata_df['name'].tolist()))
ref_name_list = [i.lower() for i in ref_name_list]
ref_email_list = list(filter(None, inputdata_df['email'].tolist()))
ref_email_list = [i.lower() for i in ref_email_list]


#give the auth token for Zoom account
auth_token = ''



authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)


if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if apac_report_datetime in str(x['start_time']):
      apac_uid = str(x['uuid'])
      print(x)
    if naol_report_datetime in str(x['start_time']):
      naol_uid = str(x['uuid'])
      print(x)



print('apac_id : '+apac_uid)
print('naol_id : '+naol_uid)


if apac_uid != "":
  apac_encoded_uid = double_encode_uuid(apac_uid)
  apac_participants_list = get_participants_list(apac_encoded_uid)
  apac_poll_answer_df = get_poll_answer_df(apac_encoded_uid)
  apac_poll1_df = get_poll1_df(apac_poll_answer_df,poll1_id)
  apac_poll2_df = get_poll2_df(apac_poll_answer_df,poll2_id)
  

if naol_uid != "":
  naol_encoded_uid = double_encode_uuid(naol_uid)
  naol_participants_list = get_participants_list(naol_encoded_uid)
  naol_poll_answer_df = get_poll_answer_df(naol_encoded_uid)
  naol_poll1_df = get_poll1_df(naol_poll_answer_df,poll1_id)
  naol_poll2_df = get_poll2_df(naol_poll_answer_df,poll2_id)
  

print(len(apac_participants_list))
print(len(naol_participants_list))

if len(apac_participants_list)>0:
  apac_participants_df = get_participants_df(apac_participants_list,'APAC',ref_id_list,ref_name_list,ref_email_list)
  #set_with_dataframe(apac_output_sheet, apac_participants_df)

if len(naol_participants_list)>0:
  naol_participants_df = get_participants_df(naol_participants_list,'NAOL',ref_id_list,ref_name_list,ref_email_list)
  #set_with_dataframe(naol_output_sheet, naol_participants_df)

final_combined_df = apac_participants_df.append(naol_participants_df)
final_combined_sheet.clear()
set_with_dataframe(final_combined_sheet, final_combined_df)

if move2side_input == 'true':
  rolling_3week_df_final = get_rolling_3week_df(rolling_3week_df,final_combined_df)
  weekend_archive_df_final = get_weekend_archive_df(weekend_archive_df,final_combined_df)

  weekend_archive_sheet.clear()
  rolling_3week_sheet.clear()
  #set_with_dataframe(rolling3week_output_sheet, rolling_3week_df_final)
  set_with_dataframe(weekend_archive_sheet,weekend_archive_df_final)
  set_with_dataframe(rolling_3week_sheet, rolling_3week_df_final)

  weekend_archive_df_final_filtered_3m_frequency = get_weekend_archive_df_final_filtered_3m(weekend_archive_df_final)
  set_with_dataframe(month_3_archive_frequency_sheet,weekend_archive_df_final_filtered_3m_frequency)

  archive_group_df_max_date_details = get_archive_group_df_max_date_details(weekend_archive_df_final)
  archive_name_list=archive_group_df_max_date_details['name'].tolist()
  archive_name_list_new = [i for i in archive_name_list if len(i) > 1]
  matched_name_list=[]
  for index,row in sideroom_data_df.iterrows():
    highest_match = process.extractOne(row['name'],archive_name_list_new)
    matched_name_list.append([row['name'],highest_match[0],highest_match[1]])

  updated_sideroom_details_df = get_updated_sideroom_details(matched_name_list,archive_group_df_max_date_details)
  
  updated_sideroom_sheet.clear()
  set_with_dataframe(updated_sideroom_sheet, updated_sideroom_details_df)
  #set_with_dataframe(updated_regular_output_sheet, updated_sideroom_details_df)

  final_max_count_filtered = pivot_rolling_3week_df(rolling_3week_df_final)
  
  regular_matched_name_list=updated_sideroom_details_df['name'].tolist()
  regular_matched_name_list_new = [i for i in regular_matched_name_list if len(i) > 1]
  move2side_name_list=[]
  for index,row in final_max_count_filtered.iterrows():
    highest_match_move2side = process.extractOne(row['name'],regular_matched_name_list_new)
    move2side_name_list.append([row['name'],highest_match_move2side[0],highest_match_move2side[1]])

  move2side_final = get_move2side_final(move2side_name_list,final_max_count_filtered)
  
  email_name_ref_df = get_email_name_ref_df(weekend_archive_df_final)
  
  move2side_final_df_with_email = pd.merge(move2side_final,email_name_ref_df,how='left',on='name')
  move2side_final_df_with_email.drop_duplicates(inplace=True)

  email_list = updated_sideroom_details_df['user_email'].to_list()
  email_list = list(filter(None, email_list))
  email_list = [i.lower() for i in email_list]

  move2side_final_df_with_email = move2side_final_df_with_email[~(move2side_final_df_with_email.user_email).str.lower().isin(email_list)]

  move2side_sheet.clear()
  set_with_dataframe(move2side_sheet, move2side_final_df_with_email)
  #set_with_dataframe(move2side_output_sheet,move2side_final_df_with_email)

with ExcelWriter(output_xlsx) as writer:
  final_combined_df.to_excel(writer, sheet_name="final_combined")
  apac_participants_df.to_excel(writer, sheet_name="APAC")
  naol_participants_df.to_excel(writer, sheet_name="NAOL")
  rolling_3week_df_final.to_excel(writer, sheet_name="3weekRollingWeekend")
  move2side_final_df_with_email.to_excel(writer, sheet_name="move2side")
  updated_sideroom_details_df.to_excel(writer, sheet_name="updated_regular_sideroom")
  apac_poll1_df.to_excel(writer, sheet_name="APAC_poll1")
  apac_poll2_df.to_excel(writer, sheet_name="APAC_poll2")
  naol_poll1_df.to_excel(writer, sheet_name="NAOL_poll1")
  naol_poll2_df.to_excel(writer, sheet_name="NAOL_poll2")

files.download(output_xlsx)